In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

In [2]:
df = pd.read_csv('Done_mbti.csv')

In [3]:
X = df['posts']
y = df['type']

In [4]:
le = LabelEncoder()
y = le.fit_transform(y)

In [5]:
max_words = 1000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

In [6]:
maxlen = max(len(x) for x in sequences)

In [7]:
padded_sequences = pad_sequences(sequences, maxlen=maxlen)

In [8]:
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_indices, test_indices = next(splitter.split(padded_sequences, y))
X_train, X_test = padded_sequences[train_indices], padded_sequences[test_indices]
y_train, y_test = y[train_indices], y[test_indices]

In [9]:
model = Sequential()
model.add(Embedding(1000, 128, input_length=maxlen))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(16, activation='softmax'))

In [10]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
217/217 [==============================] - 221s 1s/step - loss: 2.3225 - accuracy: 0.1976 - val_loss: 2.2897 - val_accuracy: 0.2092
Epoch 2/10
217/217 [==============================] - 225s 1s/step - loss: 2.2640 - accuracy: 0.2237 - val_loss: 2.2874 - val_accuracy: 0.1816
Epoch 3/10
217/217 [==============================] - 217s 1s/step - loss: 2.1510 - accuracy: 0.2904 - val_loss: 2.3350 - val_accuracy: 0.2023
Epoch 4/10
217/217 [==============================] - 217s 1s/step - loss: 2.0187 - accuracy: 0.3411 - val_loss: 2.3441 - val_accuracy: 0.2225
Epoch 5/10
217/217 [==============================] - 218s 1s/step - loss: 1.9127 - accuracy: 0.3696 - val_loss: 2.4579 - val_accuracy: 0.2046
Epoch 6/10
217/217 [==============================] - 225s 1s/step - loss: 1.8094 - accuracy: 0.3957 - val_loss: 2.6120 - val_accuracy: 0.2115
Epoch 7/10
217/217 [==============================] - 226s 1s/step - loss: 1.7010 - accuracy: 0.4382 - val_loss: 2.5559 - val_accuracy: 0.2161

In [11]:
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

55/55 [==============================] - 5s 91ms/step


In [12]:
y_true = le.inverse_transform(y_test)
y_pred = le.inverse_transform(y_pred)

In [ ]:
print("Accuracy score: ", accuracy_score(y_true, y_pred))
print("F1 score: ", f1_score(y_true, y_pred, average='weighted'))